In [1]:
# Activate Spark in our Colab notebook.
import os

# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz

#pip install both of these or else it wont work
!pip install -q findspark
!pip install pyspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/p

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-01 23:58:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.13MB/s    in 0.2s    

2023-03-01 23:58:01 (5.13 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark Dataframe

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

reviewDf = spark.read.csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

reviewDf.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
# Get the number of rows in the DataFrame.

reviewDf.count()

1767753

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = reviewDf.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])

review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R135Q3VZ4DQN5N|   50423057|B00JWXFDMG|     657335467| 2015-08-31|
|R2N0QQ6R4T7YRY|   11262325|B00W5T1H9W|      26030170| 2015-08-31|
|R3N5JE5Y4T6W5M|   27541121|B00M2L6KFY|     697845240| 2015-08-31|
|R2I150CX5IVY9Q|    5350721|B0006SW2WU|     569859289| 2015-08-31|
|R1RM9ICOOA9MQ3|   24484424|B009YPDW70|     332947422| 2015-08-31|
|R2J2KMDL10UMSH|     457767|B00QGA8XZM|     424804582| 2015-08-31|
|R3R9ZUFA4TB4FQ|   31009358|B00L0N0E8E|     589441783| 2015-08-31|
|R3UQ8VAQN7R6WL|    7034628|B00DMWQKWA|     517428445| 2015-08-31|
|R1FXZ69C01JNQM|   34608636|B00THLHAGA|     320313750| 2015-08-31|
| RY36LB4OW0FFS|   44972687|B00UXSEOHY|     589365488| 2015-08-31|
| R6PKS54PC6ZY2|   16684508|B00XGRYGVS|     217045013| 2015-08-31|
|R1LYSAH0IECVWB|   23493178|B00AJ2VU8A|     972853757| 2015-08

In [7]:
#cleaning the review_if_df just in case
review_id_df = review_id_df.dropDuplicates(["review_id"])
review_id_df = review_id_df.dropna(how='any')

# Create the Product Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = reviewDf.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JWXFDMG|Everbling Purple ...|
|B00W5T1H9W|925 Sterling Silv...|
|B00M2L6KFY|Sterling Silver C...|
|B0006SW2WU|Surgical Stainles...|
|B009YPDW70|Sterling Silver F...|
|B00QGA8XZM|CHOOSE YOUR COLOR...|
|B00L0N0E8E|Metal Factory 925...|
|B00DMWQKWA|niceEshop(TM)Clas...|
|B00THLHAGA|Genuine Natural B...|
|B00UXSEOHY|MBOX Expandable A...|
|B00XGRYGVS|Set of 3 Nose Rin...|
|B00AJ2VU8A|3-row White A Gra...|
|B00LE5Y100|KONOV Mens Stainl...|
|B013E0AOH6|TEMEGO Jewelry Me...|
|B00XAWUP96|Royal Blue/White ...|
|B00WD82F1O|10-11mm Baroque F...|
|B00JP4XCSO|Sankuwen® Fashion...|
|B00PZSM384|zjskin6688 store ...|
|B00VLMW18K|Scheppend Polymer...|
|B00U5LDBB4|Horizontal Sidewa...|
+----------+--------------------+
only showing top 20 rows



In [9]:
#cleaning the products_df just in case
products_df = products_df.dropDuplicates(["product_id"])
products_df = products_df.dropna(how='any')

# Create Customers Table

In [10]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = reviewDf.select(["customer_id"])
customers_df.show()


+-----------+
|customer_id|
+-----------+
|   50423057|
|   11262325|
|   27541121|
|    5350721|
|   24484424|
|     457767|
|   31009358|
|    7034628|
|   34608636|
|   44972687|
|   16684508|
|   23493178|
|     108294|
|   25810742|
|   12037665|
|   51793466|
|     127508|
|   38430946|
|   33684654|
|    5068631|
+-----------+
only showing top 20 rows



In [11]:
#adding count
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   45657423|    1|
|   47663761|    1|
|   11931196|    1|
|   44620216|    1|
|    1167081|    1|
|   17682867|    1|
|   10782454|    1|
|   51881951|    1|
|    1537684|    1|
|   17133428|    5|
|     661760|    1|
|    9867391|    9|
|   31268594|    1|
|   48581590|    1|
|   32466409|    3|
|   38490515|    1|
|   41318181|    4|
|   18800155|    6|
|   46306239|    1|
|   28409312|    1|
+-----------+-----+
only showing top 20 rows



# Create the Vine_Table

In [12]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = reviewDf.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|
|R2I150CX5IVY9Q|          5|            0|          0|   N|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|
|R2J2KMDL10UMSH|          5|            0|          0|   N|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|
|R1FXZ69C01JNQM|          5|            0|          0|   N|
| RY36LB4OW0FFS|          5|            0|          0|   N|
| R6PKS54PC6ZY2|          1|            4|          4|   N|
|R1LYSAH0IECVWB|          5|            0|          0|   N|
|R1UN0IZI3XQ3E2|          5|            2|          3|   N|
| RIKDXD8QGL99S|          5|            

# Load

In [16]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)